In [18]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import rasterio as rio
import os
import pandas as pd
import glob

solar_pan_type = ["above_threshold","below_threshold"]


In [19]:
final_name_list =[]

for types in solar_pan_type:
    directory_path = "Masked_images_dic\\{}".format(types)
    file_list = glob.glob(directory_path + "/*.tiff")
    file_names = [file for file in file_list]
    final_name_list.extend(file_names)

df = pd.read_csv("data\\solar_panel_analysis_dataset_final.csv")

id_list = df["GEM phase ID"].tolist()

smoothness_list = []
average_elevation_list = []

sorted_list_files = sorted(final_name_list, key=lambda x: id_list.index(x.split('_')[3].split('\\')[1]))

In [20]:

for image_file in sorted_list_files:
    dataset = rio.open(image_file)

    elevation = dataset.read(1)
    transform = dataset.transform

    x = transform[2] + np.arange(dataset.width) * transform[0]
    y = transform[5] + np.arange(dataset.height) * transform[4]
    xx, yy = np.meshgrid(x, y)


    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')


    lat_min, lat_max = np.percentile(yy.flatten(), 40), np.percentile(yy.flatten(), 60)
    lon_min, lon_max = np.percentile(xx.flatten(), 40), np.percentile(xx.flatten(), 60)
    highlight_mask = np.logical_and(yy >= lat_min, yy <= lat_max) & np.logical_and(xx >= lon_min, xx <= lon_max)

    ax.plot_surface(xx, yy, elevation, cmap='terrain', linewidth=0, alpha=0.5)

    highlight_color = 'r'

    highlighted_elevation = np.ma.masked_array(elevation, mask=~highlight_mask).astype(float)

    ax.plot_surface(xx, yy, highlighted_elevation.filled(np.nan), cmap='Reds', linewidth=0.2, alpha=1)

    ax.set_xlim(np.min(xx), np.max(xx))
    ax.set_ylim(np.min(yy), np.max(yy))
    ax.set_zlim(np.min(elevation)+5, np.max(elevation)+5)

    min_elevation_indices = np.unravel_index(np.nanargmin(elevation), elevation.shape)

    azim = - np.rad2deg(np.arctan2(min_elevation_indices[0] - yy.shape[0] // 2, min_elevation_indices[1] - xx.shape[1] // 2))


    ax.view_init(elev=30, azim=azim)

    ax.set_zlabel('Elevation')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    z_min, z_max = np.min(elevation), np.max(elevation)
    tick_distance = 10.0
    z_min = np.floor(z_min / tick_distance) * tick_distance
    z_max = np.ceil(z_max / tick_distance) * tick_distance
    num_ticks = int((z_max - z_min) / tick_distance) + 1
    z_ticks = np.linspace(z_min, z_max, num_ticks)
    ax.set_zticks(z_ticks)

    plt.savefig('Masked_images_dic\\{}\\elevation_plots\\{}.png'.format(image_file.split('\\')[1],image_file.split('\\')[-1][:-5]))

    plt.close()

    slope_x = np.gradient(elevation, axis=1)
    slope_y = np.gradient(elevation, axis=0)
    slope_magnitude = np.sqrt(slope_x ** 2 + slope_y ** 2)


    cropped_slope_magnitude = slope_magnitude[highlight_mask]
    slope_std = np.std(cropped_slope_magnitude)

    average_slope = np.mean(cropped_slope_magnitude)
    average_elevation = np.mean(elevation[highlight_mask])
    average_elevation_list.append(average_elevation)
    smoothness_list.append(average_slope)

C:\Users\cseri\AppData\Local\Temp\ipykernel_21992\746388226.py:30: UserWarning: Attempting to set identical low and high zlims makes transformation singular; automatically expanding.
  ax.set_zlim(np.min(elevation)+5, np.max(elevation)+5)


In [21]:
df['smoothness'] = smoothness_list
df['average_elevation'] = average_elevation_list

In [22]:
df.to_csv("data\\solar_panel_analysis_dataset_final.csv")